In [2]:
# Print available filepaths for this competition
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# Toggle off warnigns
import warnings
# Data frames and manipulation 
import pandas as pd
# Basic plots
from matplotlib import pyplot as plt 
# Advanced plots
import seaborn as sns
# Math operations 
import numpy as np
# Split train dataset 
from sklearn.model_selection import train_test_split 
# Test model performance 
from sklearn.metrics import accuracy_score
# Light Gradient Boosting Model Classifier
from lightgbm import LGBMClassifier
# Decicion Tree Classifier Model
from sklearn.tree import DecisionTreeClassifier
# Random Forest Model 
from sklearn.ensemble import RandomForestClassifier
# Extreme gradient boosting model
from xgboost import XGBClassifier
# Categorical Boosting Model 
from catboost import CatBoostClassifier, Pool
# KFold (Multiple model runs)
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn

# Filter out warnings
warnings.filterwarnings('ignore')

## Data Read and Processing

In [4]:
train = pd.read_csv('./playground-series-s4e6/train.csv')
test = pd.read_csv('./playground-series-s4e6/test.csv')

In [5]:
train

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


In [6]:
# Encode the target variable to numerical for further analysis
train['Target'] = train['Target'].map({'Graduate': 0, 'Dropout': 1, 'Enrolled': 2})

In [7]:
# Replace spaces in column names in he train and test sets with an underscore
train.columns = train.columns.str.replace(' ', '_', regex=True)
test.columns = test.columns.str.replace(' ', '_', regex=True)

In [8]:
# Seperate target variable
X = train.drop(columns=['Target'])
y = train['Target']

In [9]:
X_tensor = torch.tensor(X.values[:,1:], dtype=torch.float32)
print(X_tensor.shape)
y_tensor = torch.tensor(y.values)
y_one_hot = nn.functional.one_hot(y_tensor, num_classes=3)
X_train = X_tensor
y_train = y_one_hot.to(torch.float32)
print(y_train.dtype,y_train.shape)

# min_vals, _ = torch.min(X_train, dim=0, keepdim=True)
# max_vals, _ = torch.max(X_train, dim=0, keepdim=True)
# X_train = (X_train - min_vals) / (max_vals - min_vals)

torch.Size([76518, 36])
torch.float32 torch.Size([76518, 3])


In [10]:
print(row_max)
print(row_min)

NameError: name 'row_max' is not defined

In [39]:
# normalize all data to range (0,1)
row_max = X_train.max(dim=0,keepdim=True).values
row_min = X_train.min(dim=0,keepdim=True).values
X_train = (X_train-row_min)/(row_max-row_min) + 1e-5

In [40]:
X_train[0,:]

tensor([1.0000e-05, 1.0000e-05, 1.1112e-01, 9.2439e-01, 1.0000e+00, 1.0000e-05,
        3.2633e-01, 1.0000e-05, 1.0000e-05, 4.1861e-01, 2.5783e-02, 2.5651e-02,
        2.9054e-01, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00, 1.0000e-05,
        1.0000e+00, 1.8878e-02, 1.0000e-05, 1.0000e-05, 2.3078e-01, 1.3334e-01,
        2.3078e-01, 7.6822e-01, 1.0000e-05, 1.0000e-05, 2.6088e-01, 2.1213e-01,
        3.0001e-01, 6.9049e-01, 1.0000e-05, 4.0699e-01, 3.1112e-01, 8.0318e-01])

## Build Model and data loader
Previously the model was built upon the nn.transformerencoderlayer. The result was not good. Wnat to try self-implemented transformer structure

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

In [42]:
# encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
# transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
# src = torch.rand(1,512)
# out = transformer_encoder(src)
# # output = encoder_layer(src)
# print(out.shape)

In [91]:
class ClassifierModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassifierModel, self).__init__()
        self.mlp1 = nn.Linear(in_features = 36, out_features = 36)
        # self.layer1 = nn.Sequential(
        #     nn.Conv1d(in_channels=1, out_channels=8, kernel_size=19,padding = 'same',padding_mode='reflect'),
        #     nn.ReLU()
        # )
        self.relu = nn.ReLU()
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5),
            nn.ReLU()
        )
        self.mlp2 = nn.Linear(in_features=(input_dim-4*2)*32,out_features=3)
        # self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def _initialize_weights(self):
        # Initialize embedding layer
        init.xavier_uniform_(self.embedding.weight)
        if self.embedding.bias is not None:
            init.constant_(self.embedding.bias, 0)
        
        # Initialize transformer encoder layers
        for layer in self.transformer_encoder.layers:
            init.xavier_uniform_(layer.self_attention.in_proj_weight)
            init.xavier_uniform_(layer.self_attention.out_proj.weight)
            init.constant_(layer.self_attention.in_proj_bias, 0)
            init.constant_(layer.self_attention.out_proj.bias, 0)
            init.xavier_uniform_(layer.feed_forward[0].weight)
            init.xavier_uniform_(layer.feed_forward[2].weight)
            init.constant_(layer.feed_forward[0].bias, 0)
            init.constant_(layer.feed_forward[2].bias, 0)
        
        # Initialize fully connected layer with smaller values to start close to zero
        init.xavier_uniform_(self.fc.weight, gain=init.calculate_gain('sigmoid'))
        init.constant_(self.fc.bias, 0)
        
    def forward(self, x):
        x1 = self.mlp1(x)
        x1 = self.relu(x1)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        # print(x3.shape)
        x3 = self.mlp2(x3.view(-1,x3.shape[-2]*x3.shape[-1]))
        # x4 = self.relu(x3)
        output = self.softmax(x3)
        return output



In [92]:
device = 'cuda'
input_dim = 36
num_classes = 3
model = ClassifierModel(input_dim, num_classes)
model.to(device)
# summary(model, input_size=(1,37))
src = torch.rand((32,1,36), device = device)
src.to(device)
src.get_device()
output = model(src)
print(output)

tensor([[0.3275, 0.3125, 0.3600],
        [0.3259, 0.3173, 0.3568],
        [0.3293, 0.3122, 0.3585],
        [0.3211, 0.3216, 0.3573],
        [0.3261, 0.3141, 0.3598],
        [0.3260, 0.3152, 0.3589],
        [0.3269, 0.3124, 0.3607],
        [0.3259, 0.3161, 0.3580],
        [0.3269, 0.3199, 0.3532],
        [0.3262, 0.3141, 0.3597],
        [0.3251, 0.3163, 0.3586],
        [0.3268, 0.3142, 0.3591],
        [0.3232, 0.3174, 0.3595],
        [0.3251, 0.3165, 0.3584],
        [0.3281, 0.3150, 0.3569],
        [0.3237, 0.3149, 0.3614],
        [0.3273, 0.3167, 0.3560],
        [0.3305, 0.3135, 0.3560],
        [0.3271, 0.3109, 0.3620],
        [0.3253, 0.3162, 0.3585],
        [0.3252, 0.3155, 0.3594],
        [0.3280, 0.3116, 0.3605],
        [0.3269, 0.3151, 0.3580],
        [0.3208, 0.3183, 0.3609],
        [0.3308, 0.3097, 0.3596],
        [0.3276, 0.3134, 0.3589],
        [0.3292, 0.3138, 0.3570],
        [0.3264, 0.3176, 0.3559],
        [0.3280, 0.3111, 0.3609],
        [0.326

In [93]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # print(self.labels[idx].shape)
        return self.data[idx].view([-1,36]), self.labels[idx]

batch_size = 16
print(X_train.dtype, y_train.dtype)
dataset = CustomDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

train_size = int(0.9*len(dataset))
val_size =  len(dataset)-train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

torch.float32 torch.float32


## Training

In [94]:
from torchinfo import summary
from tqdm.notebook import tqdm
torch.cuda.empty_cache()
device = 'cuda'
input_dim = 36
num_classes = 3

model = ClassifierModel(input_dim, num_classes)
model.to(device)
summary(model, input_size=(2, 1, input_dim))

Layer (type:depth-idx)                   Output Shape              Param #
ClassifierModel                          [2, 3]                    --
├─Linear: 1-1                            [2, 1, 36]                1,332
├─ReLU: 1-2                              [2, 1, 36]                --
├─Sequential: 1-3                        [2, 16, 32]               --
│    └─Conv1d: 2-1                       [2, 16, 32]               96
│    └─ReLU: 2-2                         [2, 16, 32]               --
├─Sequential: 1-4                        [2, 32, 28]               --
│    └─Conv1d: 2-3                       [2, 32, 28]               2,592
│    └─ReLU: 2-4                         [2, 32, 28]               --
├─Linear: 1-5                            [2, 3]                    2,691
├─Softmax: 1-6                           [2, 3]                    --
Total params: 6,711
Trainable params: 6,711
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.16
Input size (MB): 0.00
Forward/backward

In [95]:
targets.shape

torch.Size([16, 3])

In [96]:
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
torch.autograd.set_detect_anomaly(True)

model.train()
for epoch in range(num_epochs):    
    running_loss = 0.0
    
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
            
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    # print(running_loss)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    correct_predictions = 0
    eval_loss = 0
    for inputs, targets in tqdm(val_loader):
        if len(inputs)<batch_size:
            continue
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        eval_loss += loss.item() * inputs.size(0)
        predicted_classes = torch.argmax(outputs, dim=1)
        ground_truth_classes = torch.argmax(targets, dim=1)
        correct_predictions += torch.sum(predicted_classes == ground_truth_classes).item()
    accuracy = correct_predictions / len(val_loader)/batch_size  
    eval_loss = eval_loss / len(val_loader.dataset)
    print(f'Accuracy: {accuracy}; Loss: {eval_loss}')

  0%|          | 0/4305 [00:00<?, ?it/s]

Epoch 1/10, Loss: 0.7527


  0%|          | 0/479 [00:00<?, ?it/s]

Accuracy: 0.81080375782881; Loss: 0.7335783435733794


  0%|          | 0/4305 [00:00<?, ?it/s]

...

Epoch 4/10, Loss: 0.7332


  0%|          | 0/479 [00:00<?, ?it/s]

Accuracy: 0.8126304801670147; Loss: 0.7319707714694991


  0%|          | 0/4305 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [103]:
val_loss = 0
for inputs, label in tqdm(val_loader):
    output = model(inputs.to(device))
    predicted_classes = torch.argmax(output, dim=1)
    ground_truth_classes = torch.argmax(label, dim=1)
    equal = predicted_classes.cpu() == ground_truth_classes.cpu()
    correct_predictions = torch.sum(equal).item()
    print(f'Accuracy in training set: {correct_predictions}')
    print(inputs.shape)
    loss = criterion(output, label.to('cuda'))
    val_loss+= loss.item() * inputs.size(0)
val_loss = val_loss/len(train_loader.dataset)
print(val_loss)

  0%|          | 0/3 [00:00<?, ?it/s]

Accuracy in training set: 24
torch.Size([32, 37])
Accuracy in training set: 26
torch.Size([32, 37])
Accuracy in training set: 0
torch.Size([1, 37])
0.08812510760294066
